<a href="https://colab.research.google.com/github/larguesa/FRL-CD-AM2/blob/main/FRL_CD_AM2_15_Tarefa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 15 de Aprendizagem de Máquinas II - Faculdade de Tecnologia de Santos
## Busca Semântica com RAG (Retrieval-Augmented Generation)

Este notebook demonstra como usar a API do Gemini para realizar buscas semânticas com dados de uma empresa fictícia, "NexusTech". Vamos:

1. Criar uma lista de 10 perguntas e respostas sobre uma empresa fictícia
2. Gerar embeddings para as respostas usando a API do Gemini e armazená-los em memória.
3. Implementar uma função de busca semântica para encontrar pares pergunta-resposta relevantes com base em uma query.
4. Gerar respostas para queries de exemplo usando o contexto recuperado.

**Pré-requisitos:**
- Uma chave de API do Gemini (inclua sua chave no secrets do Colab no ícone da chave do menu lateral esquerdo).
- Instalar as dependências necessárias.

In [ ]:
# Célula 1: Instalação e importação de bibliotecas
!pip install -q google-generativeai sklearn numpy

import google.generativeai as genai
from google.colab import userdata
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import textwrap

# Função auxiliar para formatar markdown (opcional, para melhor visualização)
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

print("Bibliotecas instaladas e importadas.")

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Bibliotecas instaladas e importadas.


In [ ]:
# Célula 2: Configuração da API Key do Gemini
# NOTA: Vá em "Secrets" (ícone de chave no menu à esquerda do Colab)
# e adicione sua API Key com o nome "GOOGLE_API_KEY".
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("API Key do Gemini configurada com sucesso!")
except userdata.SecretNotFoundError:
    print("Erro: Chave 'GOOGLE_API_KEY' não encontrada nos Secrets do Colab.")
    print("Por favor, adicione sua API Key do Gemini nos Secrets com o nome 'GOOGLE_API_KEY'.")
    # Você pode adicionar um input manual como fallback se preferir, mas Secrets é mais seguro:
    # GOOGLE_API_KEY = input("Cole sua API Key do Gemini aqui: ")
    # genai.configure(api_key=GOOGLE_API_KEY)
except Exception as e:
    print(f"Ocorreu um erro ao configurar a API Key: {e}")

API Key do Gemini configurada com sucesso!


In [ ]:
# Célula 3: Definição dos dados da empresa fictícia (Perguntas e Respostas)

dados_empresa_ficticia = [
    {
        "id": "qa_01",
        "pergunta": "Qual o nome da empresa?",
        "resposta": "O nome da empresa é NexusTech Soluções Inovadoras."
    },
    {
        "id": "qa_02",
        "pergunta": "Quando a NexusTech foi fundada?",
        "resposta": "A NexusTech foi fundada em 15 de março de 2010."
    },
    {
        "id": "qa_03",
        "pergunta": "Quem são os fundadores da NexusTech?",
        "resposta": "Os fundadores da NexusTech são Ana Silva (CEO), Bruno Costa (CTO) e Carla Dias (COO)."
    },
    {
        "id": "qa_04",
        "pergunta": "Quantos funcionários a NexusTech possui atualmente?",
        "resposta": "A NexusTech possui atualmente cerca de 150 funcionários."
    },
    {
        "id": "qa_05",
        "pergunta": "Qual é o principal serviço de desenvolvimento oferecido pela NexusTech?",
        "resposta": "O principal serviço de desenvolvimento da NexusTech é o desenvolvimento de software sob medida para as necessidades específicas de cada cliente."
    },
    {
        "id": "qa_06",
        "pergunta": "A NexusTech oferece serviços de consultoria?",
        "resposta": "Sim, a NexusTech oferece consultoria especializada em transformação digital para empresas de diversos portes."
    },
    {
        "id": "qa_07",
        "pergunta": "Como funciona o suporte técnico da NexusTech?",
        "resposta": "A NexusTech oferece suporte técnico especializado 24 horas por dia, 7 dias por semana, para garantir a continuidade das operações de seus clientes."
    },
    {
        "id": "qa_08",
        "pergunta": "Qual é a missão da NexusTech?",
        "resposta": "A missão da NexusTech é empoderar negócios através de tecnologia de ponta e soluções personalizadas."
    },
    {
        "id": "qa_09",
        "pergunta": "Qual a visão de futuro da NexusTech?",
        "resposta": "A visão da NexusTech é ser líder global em inovação tecnológica, sendo reconhecida pela excelência e pelo impacto positivo que gera em seus clientes e na sociedade."
    },
    {
        "id": "qa_10",
        "pergunta": "Quais são os principais valores da NexusTech?",
        "resposta": "Os principais valores da NexusTech são: Inovação contínua, Colaboração entre equipes e com clientes, foco no Cliente no Centro de todas as decisões, Integridade em todas as ações e busca incessante pela Excelência."
    }
]

# Combinar pergunta e resposta para vetorização
textos_para_vetorizar = [f"Pergunta: {item['pergunta']} Resposta: {item['resposta']}" for item in dados_empresa_ficticia]

print(f"{len(textos_para_vetorizar)} Q&As definidos para a NexusTech Soluções Inovadoras.")

10 Q&As definidos para a NexusTech Soluções Inovadoras.


In [ ]:
# Célula 4: Vetorização dos dados (Q&A) usando a API do Gemini

# Modelo de embedding
embedding_model = 'models/embedding-001' # Modelo mais recente pode ser text-embedding-004

embeddings_list = []
print("Iniciando vetorização dos Q&As...")
try:
    for i, texto in enumerate(textos_para_vetorizar):
        print(f"Vetorizando Q&A {i+1}/{len(textos_para_vetorizar)}...")
        # O retry automático já está embutido na biblioteca Python a partir de certas versões
        # Se encontrar erros de quota, pode ser necessário adicionar um time.sleep()
        response = genai.embed_content(model=embedding_model,
                                       content=texto,
                                       task_type="RETRIEVAL_DOCUMENT") # ou "SEMANTIC_SIMILARITY"
        embeddings_list.append(response['embedding'])
    embeddings_qa = np.array(embeddings_list)
    print(f"Vetorização concluída! Dimensão dos embeddings: {embeddings_qa.shape}")
except Exception as e:
    print(f"Erro durante a vetorização: {e}")
    print("Verifique sua API Key, cotas e o modelo de embedding especificado.")
    embeddings_qa = None # Define como None para evitar erros subsequentes

Iniciando vetorização dos Q&As...
Vetorizando Q&A 1/10...
Vetorizando Q&A 2/10...
Vetorizando Q&A 3/10...
Vetorizando Q&A 4/10...
Vetorizando Q&A 5/10...
Vetorizando Q&A 6/10...
Vetorizando Q&A 7/10...
Vetorizando Q&A 8/10...
Vetorizando Q&A 9/10...
Vetorizando Q&A 10/10...
Vetorização concluída! Dimensão dos embeddings: (10, 768)


In [ ]:
# Célula 5: Função para busca por similaridade semântica

def buscar_contextos_similares(pergunta_usuario, embeddings_base, textos_base, top_n=2):
    if embeddings_base is None or len(embeddings_base) == 0:
        print("Embeddings base não estão disponíveis.")
        return [], []

    print(f"\nBuscando contextos para a pergunta: '{pergunta_usuario}'")
    # Vetorizar a pergunta do usuário
    try:
        embedding_pergunta = genai.embed_content(model=embedding_model,
                                                 content=pergunta_usuario,
                                                 task_type="RETRIEVAL_QUERY")['embedding'] # ou "SEMANTIC_SIMILARITY"
    except Exception as e:
        print(f"Erro ao vetorizar a pergunta do usuário: {e}")
        return [], []

    embedding_pergunta_np = np.array(embedding_pergunta).reshape(1, -1)

    # Calcular similaridade de cosseno
    similaridades = cosine_similarity(embedding_pergunta_np, embeddings_base)

    # Obter os índices dos top_n mais similares
    # argsort retorna os índices que ordenariam o array. Pegamos os últimos N.
    indices_similares = similaridades[0].argsort()[-top_n:][::-1] # Do mais similar para o menos

    contextos_encontrados_texto = [textos_base[i] for i in indices_similares]
    ids_contextos_originais = [dados_empresa_ficticia[i]['id'] for i in indices_similares]

    print(f"Top {top_n} contextos encontrados (IDs): {ids_contextos_originais}")
    return contextos_encontrados_texto, ids_contextos_originais

In [ ]:
# Célula 6: Função para gerar resposta usando o Gemini com contexto

# Modelo generativo
generation_model = genai.GenerativeModel('gemini-1.5-flash-latest') # ou 'gemini-pro'

def gerar_resposta_com_contexto(pergunta_usuario, contextos):
    if not contextos:
        prompt_final = f"""Responda à seguinte pergunta da melhor forma possível, com base no seu conhecimento geral,
        já que nenhum contexto específico foi fornecido:

        Pergunta: {pergunta_usuario}

        Resposta:
        """
    else:
        contexto_formatado = "\n\n".join([f"Contexto {i+1}:\n{ctx}" for i, ctx in enumerate(contextos)])
        prompt_final = f"""Com base nos seguintes contextos:

{contexto_formatado}

Responda à pergunta: "{pergunta_usuario}"

---
Instruções Adicionais:
- Use APENAS as informações dos contextos fornecidos para responder à pergunta.
- Se a resposta não estiver nos contextos, diga explicitamente que a informação não foi encontrada nos documentos fornecidos.
- Não invente informações.

Resposta:
"""

    print("\n--- Prompt Enviado ao Modelo Generativo ---")
    print(prompt_final)
    print("----------------------------------------")

    try:
        resposta_gemini = generation_model.generate_content(prompt_final)
        return resposta_gemini.text
    except Exception as e:
        return f"Erro ao gerar resposta: {e}"

In [ ]:
# Célula 7: Execução dos exemplos

perguntas_exemplo = [
    "Quem é o CEO da NexusTech?",
    "Quantos funcionários a NexusTech possui atualmente?",
    "O que a NexusTech faz?",
    "Qual é o principal objetivo da empresa NexusTech?",
    "Como a NexusTech ajuda outras empresas a se modernizarem?",
    "A empresa oferece algum tipo de garantia ou acordo de nível de serviço (SLA) para o suporte técnico?" # Pergunta que pode não estar explicitamente nos Q&As
]

if embeddings_qa is not None and GOOGLE_API_KEY: # Verifica se a vetorização e a API Key estão ok
    for pergunta_user in perguntas_exemplo:
        print(f"\n======================================================================")
        print(f"PROCESSANDO PERGUNTA DO USUÁRIO: {pergunta_user}")
        print(f"======================================================================")

        contextos_selecionados, ids_contextos = buscar_contextos_similares(
            pergunta_user,
            embeddings_qa,
            textos_para_vetorizar,
            top_n=2  # Você pode ajustar o número de contextos a serem recuperados
        )

        print("\n--- Contextos que serão incorporados ao prompt ---")
        if contextos_selecionados:
            for i, ctx_text in enumerate(contextos_selecionados):
                # Encontrar o Q&A original correspondente para exibição mais clara
                qa_original = next(item for item in dados_empresa_ficticia if item["id"] == ids_contextos[i])
                print(f"ID do Q&A: {qa_original['id']}")
                print(f"  P: {qa_original['pergunta']}")
                print(f"  R: {qa_original['resposta']}\n")
        else:
            print("Nenhum contexto similar encontrado para esta pergunta.")
        print("----------------------------------------------------")

        resposta_final = gerar_resposta_com_contexto(pergunta_user, contextos_selecionados)

        print("\n--- Resposta Final do Gemini ---")
        print(textwrap.fill(resposta_final, width=100)) # Formata a largura da linha para melhor leitura
        print("--------------------------------\n")
        if contextos_selecionados:
            print(f"Contextos utilizados para esta resposta (IDs): {ids_contextos}")
        else:
            print("Nenhum contexto específico foi utilizado.")
        print("--------------------------------\n")

else:
    if not GOOGLE_API_KEY:
        print("A API Key do Gemini não foi configurada. Execute a Célula 2 e configure-a.")
    if embeddings_qa is None:
        print("A vetorização dos Q&As falhou ou não foi executada. Verifique a Célula 4.")


PROCESSANDO PERGUNTA DO USUÁRIO: Quem é o CEO da NexusTech?

Buscando contextos para a pergunta: 'Quem é o CEO da NexusTech?'
Top 2 contextos encontrados (IDs): ['qa_03', 'qa_09']

--- Contextos que serão incorporados ao prompt ---
ID do Q&A: qa_03
  P: Quem são os fundadores da NexusTech?
  R: Os fundadores da NexusTech são Ana Silva (CEO), Bruno Costa (CTO) e Carla Dias (COO).

ID do Q&A: qa_09
  P: Qual a visão de futuro da NexusTech?
  R: A visão da NexusTech é ser líder global em inovação tecnológica, sendo reconhecida pela excelência e pelo impacto positivo que gera em seus clientes e na sociedade.

----------------------------------------------------

--- Prompt Enviado ao Modelo Generativo ---
Com base nos seguintes contextos:

Contexto 1:
Pergunta: Quem são os fundadores da NexusTech? Resposta: Os fundadores da NexusTech são Ana Silva (CEO), Bruno Costa (CTO) e Carla Dias (COO).

Contexto 2:
Pergunta: Qual a visão de futuro da NexusTech? Resposta: A visão da NexusTech é ser l

### Como Usar o Notebook:

1.  **Adicione sua API Key do Gemini:**
    * No menu à esquerda do Colab, clique no ícone de chave ("Secrets").
    * Clique em "+ Add a new secret".
    * Nomeie o segredo como `GOOGLE_API_KEY`.
    * Cole sua API Key do Gemini no campo "Value".
    * Ative o botão "Notebook access".
2.  **Execute as Células:** Execute as células em ordem, uma por uma (Shift + Enter ou clicando no botão de play).
    * A **Célula 1** instala as bibliotecas.
    * A **Célula 2** configura sua API Key.
    * A **Célula 3** define os dados da empresa fictícia.
    * A **Célula 4** vetoriza esses dados usando o modelo de embedding do Gemini. Isso pode levar alguns segundos.
    * A **Célula 5** define a função para busca semântica.
    * A **Célula 6** define a função para gerar a resposta final com contexto.
    * A **Célula 7** executa o processo para algumas perguntas de exemplo, mostrando os contextos selecionados e a resposta gerada.

### Observações:

* **Custos da API:** Lembre-se de que o uso da API do Gemini (tanto para embeddings quanto para geração de texto) pode incorrer em custos, dependendo do seu volume de uso e das cotas da sua conta.
* **Modelos:** Usei `models/embedding-001` para embeddings e `gemini-1.5-flash-latest` para geração. Você pode experimentar outros modelos disponíveis. O `text-embedding-004` é uma opção mais recente para embeddings.
* **Similaridade:** A busca por similaridade aqui usa a similaridade de cosseno, que é uma métrica comum para comparar embeddings.
* **Número de Contextos (`top_n`):** Você pode ajustar o parâmetro `top_n` na função `buscar_contextos_similares` para controlar quantos dos Q&As mais relevantes são passados como contexto para o modelo generativo.